In [26]:
%%writefile src/Multiply7Event.sol
pragma solidity 0.5.1;
contract Multiply7Event {
    event Print(address from, uint256 timestamp, uint256 result);
    function multiply(uint256 input) public {
        emit Print(msg.sender, now, input*7);
    }
}

Overwriting src/Multiply7Event.sol


In [27]:
!solc --abi --bin --gas src/Multiply7Event.sol


======= src/Multiply7Event.sol:Multiply7Event =======
Gas estimation:
construction:
   105 + 56400 = 56505
external:
   multiply(uint256):	1854
Binary: 
608060405234801561001057600080fd5b5061011a806100206000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063c6888fa114603e575b600080fd5b348015604957600080fd5b50607360048036036020811015605e57600080fd5b81019080803590602001909291905050506075565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334260078402604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea165627a7a723058208cc1a6558c44fd2cbada4e6b779166fd0393819034a6a06ba8b8e335960740930029
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexe

In [28]:
%%writefile src/Multiply7EventDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"timestamp","type":"uint256"},{"indexed":false,"name":"result","type":"uint256"}],"name":"Print","type":"event"}];
var _bin="0x"+"608060405234801561001057600080fd5b5061011a806100206000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063c6888fa114603e575b600080fd5b348015604957600080fd5b50607360048036036020811015605e57600080fd5b81019080803590602001909291905050506075565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334260078402604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea165627a7a723058208cc1a6558c44fd2cbada4e6b779166fd0393819034a6a06ba8b8e335960740930029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Multiply7EventDeploy.js


In [29]:
!node src/Multiply7EventDeploy.js

Deploying the contract from 0x38B1F38F5D033a37e3710B382ED0140285ED1E60
hash: 0x1f9f37556bae9cfe58195ca4ae54a6be129f471e21047ab490247324a5649019
---> The contract deployed to: 0x46a16dc3d1Ae8EF6C51147010578530B809fc849


In [30]:
%%writefile src/Multiply7EventWsUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"timestamp","type":"uint256"},{"indexed":false,"name":"result","type":"uint256"}],"name":"Print","type":"event"}];
var _test = new web3.eth.Contract(_abiArray, '0x46a16dc3d1Ae8EF6C51147010578530B809fc849');
var event = _test.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const value8 = await _test.methods.multiply(8)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log("---> multiply(8) called "+ JSON.stringify(value8.events.Print.returnValues));
    
    const value16 = await _test.methods.multiply(16)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log("---> multiply(16) called "+ JSON.stringify(value16.events.Print.returnValues));
}
doIt()

Overwriting src/Multiply7EventWsUse.js


In [31]:
!node src/Multiply7EventWsUse.js

Account: 0x38B1F38F5D033a37e3710B382ED0140285ED1E60
Event fired: {"logIndex":0,"transactionIndex":0,"transactionHash":"0x22197584d93cc7fba05514862d72edec13b0128188af2942e4d36a4d017bdd34","blockHash":"0xc48dea1b033d7fe939609ed591a2d4528b992742150b8fddabd74f427227bf35","blockNumber":2,"address":"0x46a16dc3d1Ae8EF6C51147010578530B809fc849","type":"mined","id":"log_7106416d","returnValues":{"0":"0x38B1F38F5D033a37e3710B382ED0140285ED1E60","1":"1591427801","2":"56","from":"0x38B1F38F5D033a37e3710B382ED0140285ED1E60","timestamp":"1591427801","result":"56"},"event":"Print","signature":"0x91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205","raw":{"data":"0x00000000000000000000000038b1f38f5d033a37e3710b382ed0140285ed1e60000000000000000000000000000000000000000000000000000000005edb42d90000000000000000000000000000000000000000000000000000000000000038","topics":["0x91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205"]}}
---> {"0":"0x38B1F38F5D033a37e3710B382ED0140285ED1E60